In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import json  
import codecs
import pandas as pd
import os
import shutil
from datetime import datetime, timedelta, date
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.options import Options
from os import remove
import git
import datetime
import numpy as np
import wget
import http.client, urllib.request, urllib.parse, urllib.error, base64




data_confirmed = pd.read_csv("time_series_covid19_confirmed_global.csv")
data_recovered = pd.read_csv("time_series_covid19_recovered_global.csv")
data_death     = pd.read_csv("time_series_covid19_deaths_global.csv")

data_salida_confirmed = pd.DataFrame() 
data_salida_recovered = pd.DataFrame()
data_salida_death = pd.DataFrame()
data_salida_confirmed["Province/State"] = data_confirmed["Province/State"] 
data_salida_confirmed["Country/Region"] = data_confirmed["Country/Region"] 
data_salida_confirmed["Lat"] = data_confirmed["Lat"]
data_salida_confirmed["Long"] = data_confirmed["Long"]

data_salida_recovered["Country/Region"] = data_recovered["Country/Region"] 
data_salida_recovered["Province/State"] = data_recovered["Province/State"] 
data_salida_death["Country/Region"] = data_death["Country/Region"]

columna_anterior = ""
for columna in data_confirmed.columns[4:len(data_confirmed)]:
    if(columna_anterior == ""):
        #print("no se hace nada")
        pass
    else:
        #print(columna + " y " + columna_anterior)

        data_salida_confirmed[columna] = data_confirmed[columna] - data_confirmed[columna_anterior]
        data_salida_recovered[columna] = data_recovered[columna] - data_recovered[columna_anterior]
        data_salida_death[columna]     = data_death[columna]     - data_death[columna_anterior]

    columna_anterior = columna

entrada = {"Province/State":""}   #,"Country/Region":"","Lat":"","Long":""}
salida = []

data_salida = pd.DataFrame(columns=data_salida_confirmed.columns[:4])

lista_confirmed = list(data_salida_confirmed.iterrows())
lista_recovered = list(data_salida_recovered.iterrows())
lista_death = list(data_salida_death.iterrows())

n = 0
for i in range(len(lista_confirmed)):
    dias_correlativo = 0
    flag = True
    for columna in data_salida_confirmed.columns[4:]:         
        #print(columna)
        #print(i[1][columna])
        entrada["Province/State"] = lista_confirmed[i][1]["Province/State"]
        entrada["Country/Region"] = lista_confirmed[i][1]["Country/Region"]

        entrada["Lat"] = lista_confirmed[i][1]["Lat"]
        entrada["Long"] = lista_confirmed[i][1]["Long"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.datetime(int(formato[2]+"20"),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
        #print(lista_confirmed[i][1][columna]) 
        #print(entrada["Country/Region"])
        entrada["confirmados"] = lista_confirmed[i][1][columna]
        entrada["fallecidos"] = lista_death[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]

        if(flag):
            if(entrada["confirmados"] != 0):
                flag = False;
                dias_correlativo += 1
        else:
            dias_correlativo += 1
        entrada["dias correlativo"] = dias_correlativo
        salida.append(data_salida.append(entrada,ignore_index=True))
data_salida = pd.concat(salida)

entrada = {"Province/State":""}
salida = []
data_salida_aux = pd.DataFrame(columns=data_salida_confirmed.columns[:4])
for i in range(len(lista_recovered)):
    for columna in data_salida_recovered.columns[4:]:
        entrada["Province/State"] = lista_recovered[i][1]["Province/State"]
        entrada["Country/Region"] = lista_recovered[i][1]["Country/Region"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.date(int(formato[2]+"20"),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
        entrada["recuperado"] = lista_recovered[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        #print(entrada)
        #print("*****************************************************************")
        salida.append(data_salida_aux.append(entrada, ignore_index=True))
data_salida_aux = pd.concat(salida)
del data_salida_aux["Province/State"]
del data_salida_aux["Country/Region"]
del data_salida_aux["fecha"]
del data_salida_aux["Lat"]
del data_salida_aux["Long"]

merged_left = pd.merge(left=data_salida, right=data_salida_aux, how='left', left_on='codigo', right_on='codigo')

In [4]:
data_salida_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20',
       '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20',
       '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20',
       '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20',
       '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20',
       '4/9/20', '4/10/20', '4/11/20'